# ReOpt

- API structure: https://developer.nrel.gov/api/reopt/stable/help?API_KEY=DEMO_KEY
- Models.py: https://github.com/NREL/REopt_API/blob/master/reoptjl/models.py

In [2]:
import os, sys, json
import shutil
import time
import yaml
import schedule
import pandas as pd
from src.post_and_poll import get_api_results
from utils import import_df, import_json, export_json, plotly_stacked, parse_dispatch_series, setup

# to silence warnigns
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

pd.options.plotting.backend='plotly'

API_KEY = '0bgdMrMpcFEfzIuYLRFpEfPeUqabOQKo5RagyRPV' 

In [3]:
def create_post(cfg,array,solar_kw,batt_kw,batt_h,solar_kw_range=False,batt_kw_range=False,batt_h_range=False):
    post = {}
    post['Site'] = cfg.Site
    post['PV'] = cfg.PV
    post['ElectricLoad'] = cfg.ElectricLoad
    post['ElectricStorage'] = cfg.ElectricStorage
    post['ElectricTariff'] = cfg.ElectricTariff
    post['ElectricUtility'] = cfg.ElectricUtility
    post['Financial'] = cfg.Financial
    
    post['PV'].update({ "min_kw":solar_kw,
                        "max_kw":solar_kw,            
                        "production_factor_series":import_df(cfg.solar_file,array,
                                                             resamp=cfg.solar_resamp,
                                                             scale=cfg.solar_scaler)})
    if solar_kw_range:
        post['PV'].update({ "min_kw":0})
    
    post['ElectricLoad'].update({'loads_kw':import_df(cfg.load_file,cfg.load_col,
                                                      resamp=cfg.load_resamp)})
            
    post['ElectricStorage'].update({
            "min_kw":batt_kw,               "max_kw":batt_kw,
            "min_kwh":batt_kw*batt_h,       "max_kwh":batt_kw*batt_h,})
    if batt_kw_range:
        post['ElectricStorage'].update({"min_kw":0})
    if batt_h_range:
        post['ElectricStorage'].update({"min_kwh":0})    

    post["ElectricTariff"].update({
        "urdb_response":import_json(cfg.tariff_file),
        'wholesale_rate':[cfg.energy_price_sell_constant]*8760,
        #'wholesale_rate':import_df('data/PG&E NBT EEC Values 2024 Vintage.csv','Price',resamp='1h'),
    })
    
    export_json(post,cfg.outdir)
    
    return post

def run_reopt(post, print_results=False):

    outputs_file_name = "results_file"
    root_url = "https://developer.nrel.gov/api/reopt/stable" # /stable == /v3 
    
    try:
        api_response = get_api_results(post=post, 
                                    API_KEY=API_KEY, 
                                    api_url=root_url, 
                                    results_file= f'outputs/{outputs_file_name}.json', 
                                    run_id=None)
    except:
        print('API request failed')
        
    if api_response is not None:
        cost =      api_response["outputs"]["ElectricTariff"]["year_one_energy_cost_before_tax"]
        revenue =   api_response["outputs"]["ElectricTariff"]["year_one_export_benefit_before_tax"]
        netcost =   cost - revenue
        npv =       api_response['outputs']['Financial']['npv']
        spp =       api_response['outputs']['Financial']['simple_payback_years']

        if print_results:    
            print('Status = ',              api_response["status"])
            print("Energy cost ($) = ",     cost)
            print('Energy revenue ($) = ',  revenue)
            print('Net cost ($) = ',        netcost)
            print('NPV ($) and Payback Period (y) = ',        npv, spp)
            print('PV Size (kW) = ',        api_response["outputs"]["PV"]["size_kw"])
            if "ElectricStorage" in api_response["outputs"].keys():
                print('Storage Size (kW-kwh) = ',api_response["outputs"]["ElectricStorage"]["size_kw"],'-',api_response["outputs"]["ElectricStorage"]["size_kwh"])
    else:
        print('API request failed')
        cost,netcost = pd.NA,pd.NA
        
    return api_response, cost, netcost
        
def compare_to_s20(cfg,solar_kw:int,batt_kw:int,batt_h:int,print_results=False):
    cost_red,cost_red_pct,netcost_red = [], [], []
    post = create_post(cfg,'0w90',solar_kw,batt_kw,batt_h)
    _, cost_base, netcost_base = run_reopt(post,print_results)
    for array in cfg.arrays:
        try:
            post = create_post(cfg,f'{array}w90',solar_kw,batt_kw,batt_h)
            _, cost, netcost = run_reopt(post,print_results)
            cost_red.append(        cost_base - cost)
            cost_red_pct.append(    1 - cost/abs(cost_base))
            netcost_red.append(     netcost_base - netcost)
            if print_results:
                print(f'\nCost Reduction {array} ($) = {cost_base-cost:.2f} ({100*(1-cost/abs(cost_base)):.1f}%)')
                print(f'NetCost Reduction {array} ($) = {netcost_base-netcost:.2f} ({100*(1-netcost/abs(netcost_base)):.1f}%)\n')
        except:
            cost_red.append(        pd.NA)
            cost_red_pct.append(    pd.NA)
            netcost_red.append(     pd.NA)
    return cost_base, cost_red, cost_red_pct, netcost_red

# Setup

In [44]:
cfg = setup('reopt.yaml')

# Single run

In [45]:
post = create_post(cfg,'0w90',200,0,0,solar_kw_range=True)
api_response, cost, netcost = run_reopt(post,print_results=True)
print(cost, netcost)

Status =  optimal
Energy cost ($) =  7626.91
Energy revenue ($) =  3392.0
Net cost ($) =  4234.91
NPV ($) and Payback Period (y) =  99971.74 8.36
PV Size (kW) =  123.0103
7626.91 4234.91


In [46]:
post = create_post(cfg,'0w90',200,100,2,solar_kw_range=True,batt_kw_range=True,batt_h_range=True)
api_response, cost, netcost = run_reopt(post,print_results=True)
print(cost, netcost)

Status =  optimal
Energy cost ($) =  3592.14
Energy revenue ($) =  2002.0
Net cost ($) =  1590.1399999999999
NPV ($) and Payback Period (y) =  181081.72 5.74
PV Size (kW) =  131.5073
Storage Size (kW-kwh) =  48.1 - 155.62
3592.14 1590.1399999999999


In [47]:
post = create_post(cfg,'50w90',200,0,0)
api_response, cost, netcost = run_reopt(post,print_results=True)
print(cost, netcost)

Status =  optimal
Energy cost ($) =  5729.32
Energy revenue ($) =  6977.0
Net cost ($) =  -1247.6800000000003
NPV ($) and Payback Period (y) =  94595.59 13.25
PV Size (kW) =  200.0
5729.32 -1247.6800000000003


In [48]:
post = create_post(cfg,'50w90',200,100,2)
api_response, cost, netcost = run_reopt(post,print_results=True)
print(cost, netcost)

Status =  optimal
Energy cost ($) =  1427.87
Energy revenue ($) =  4695.0
Net cost ($) =  -3267.13
NPV ($) and Payback Period (y) =  118601.7 10.85
PV Size (kW) =  200.0
Storage Size (kW-kwh) =  100.0 - 200.0
1427.87 -3267.13


# Grid search

In [5]:
cols = ['SolarKW','BattKW','BattH','BaseCost']
df = pd.DataFrame([],columns=cols   + [f'Reduc_{x}w90' for x in cfg.arrays]\
                                    + [f'Reduc%_{x}w90' for x in cfg.arrays]\
                                    + [f'NetReduc_{x}w90' for x in cfg.arrays])

#best_cost_red_pct = 0
#best_resp = None

if cfg.test:
    cfg.solar_kws   = [cfg.solar_kws[0]]
    cfg.batt_kws    = [cfg.batt_kws[0]]
    cfg.batt_hs     = [cfg.batt_hs[0]]

for solar_kw in cfg.solar_kws:
    for batt_kw in cfg.batt_kws:
        for batt_h in cfg.batt_hs:
            basecost, cost_red, cost_red_pct, netcost_red = compare_to_s20(cfg,solar_kw,batt_kw,batt_h,print_results=False)
            df.loc[len(df)] = [solar_kw,batt_kw,batt_h,basecost]+cost_red+cost_red_pct+netcost_red
            df.to_csv(cfg.outdir+'results.csv',index=False)
            print(df.iloc[:,:7])
            
            # for x,r,n in zip(cost_red_pct,resp,arrays):
            #     if x > best_cost_red_pct:
            #         best_cost_red_pct = x
            #         best_resp = r
            #         df.iloc[-1,:].to_csv(f'outputs/best_result_{n}.csv')
            
df

   SolarKW  BattKW  BattH  BaseCost  Reduc_25w90  Reduc%_25w90  NetReduc_25w90
0    100.0     0.0    1.0   8314.78       152.21      0.018306         -140.79
   SolarKW  BattKW  BattH  BaseCost  Reduc_25w90  Reduc%_25w90  NetReduc_25w90
0    100.0     0.0    1.0   8314.78       152.21      0.018306         -140.79
1    100.0     0.0    2.0   8314.78       152.21      0.018306         -140.79
   SolarKW  BattKW  BattH  BaseCost  Reduc_25w90  Reduc%_25w90  NetReduc_25w90
0    100.0     0.0    1.0   8314.78       152.21      0.018306         -140.79
1    100.0     0.0    2.0   8314.78       152.21      0.018306         -140.79
2    100.0    25.0    1.0   7339.73       150.33      0.020482         -148.67
API request failed


UnboundLocalError: cannot access local variable 'api_response' where it is not associated with a value

# Plot

In [ ]:
# dispatch = parse_dispatch_series(api_response)
# f = plotly_stacked(dispatch,solar='pv',load='load',batt='batt',utility='grid',soc='soc',theme='plotly_dark')

# Schedule

In [6]:
def job(tariff_name):
    try:
        with open('tariff_gs.yaml', 'r') as f:
            d=yaml.safe_load(f)
            
            d['tariff_file'] = f'tariff/{tariff_name}.json'
            
        with open('tariff_gs.yaml', 'w') as f:
            yaml.dump(d, f)
            
        cfg = setup('tariff_gs.yaml')
        
        cols = ['SolarKW','BattKW','BattH','BaseCost']
        df = pd.DataFrame([],columns=cols   + [f'Reduc_{x}w90' for x in cfg.arrays]\
                                            + [f'Reduc%_{x}w90' for x in cfg.arrays]\
                                            + [f'NetReduc_{x}w90' for x in cfg.arrays])
        if cfg.test:
            cfg.solar_kws   = [cfg.solar_kws[0]]
            cfg.batt_kws    = [cfg.batt_kws[0]]
            cfg.batt_hs     = [cfg.batt_hs[0]]

        for solar_kw in cfg.solar_kws:
            for batt_kw in cfg.batt_kws:
                for batt_h in cfg.batt_hs:
                    basecost, cost_red, cost_red_pct, netcost_red = compare_to_s20(cfg,solar_kw,batt_kw,batt_h,print_results=False)
                    df.loc[len(df)] = [solar_kw,batt_kw,batt_h,basecost]+cost_red+cost_red_pct+netcost_red
                    df.to_csv(cfg.outdir+'results.csv',index=False)
                    print(df.iloc[:,:7])
    except:
        pass

In [5]:
schedule.every().day.at('01:03').do(job,tariff_name='h4_6-10')
schedule.every().day.at('02:03').do(job,tariff_name='h4_8-12')
schedule.every().day.at('03:03').do(job,tariff_name='h4_10-14')
schedule.every().day.at('04:03').do(job,tariff_name='h4_12-16')
schedule.every().day.at('05:03').do(job,tariff_name='h4_14-18')
schedule.every().day.at('06:03').do(job,tariff_name='h4_16-20')
schedule.every().day.at('07:03').do(job,tariff_name='h8_12-20')
schedule.every().day.at('08:03').do(job,tariff_name='h8_14-22')
schedule.every().day.at('09:03').do(job,tariff_name='h8_16-24')
schedule.every().day.at('10:03').do(job,tariff_name='h12_8-20')
schedule.every().day.at('11:03').do(job,tariff_name='h12_10-22')
schedule.every().day.at('12:03').do(job,tariff_name='h12_12-24')

while True:
    schedule.run_pending()
    time.sleep(1)

FileNotFoundError: [Errno 2] No such file or directory: 'data/tariff/h4_8-10.json'